### TF Differentiation of Ipsen upper bound formula

Testing numerically: Is the matrix-vector multiplication norm a convex function? Or a concave function? Or indefinite?

The problem seems to be concave (in conic optimization terms, which needs to be a minimization problem). We cannot define a convex set that bounds our desired objective value. Generally the zero-one problem was NP-hard, but now we know that a quadratic, continuous approximation may also be NP-hard (and the shape of the convex cone would push us to the extrema towards nearly integer solutions, so there is not an easy and efficient way to solve it).

In [2]:
import numpy as np
np.set_printoptions(linewidth = 150, precision = 4, suppress = True)

In [13]:
data = np.loadtxt('../NC-Data.csv', delimiter=',', dtype=str)
data = data[1:].astype(np.float64)
# Only take 10 columns for testing.
np.random.seed(4)
def sample_hermitian(mat, choices):
    return mat[:, choices][choices, :]
data = sample_hermitian(data, np.random.choice(data.shape[0], 10))
# Synthesize columns holding observations, reproducing the cov matrix.
raw_data = np.linalg.cholesky(data).T
np.testing.assert_allclose(data, raw_data.T @ raw_data)
data

array([[ 1.    , -0.3093, -0.593 ,  0.2698,  0.0679,  0.349 , -0.7985, -0.1988, -0.0798, -0.0217],
       [-0.3093,  1.    ,  0.3678,  0.1036,  0.0957, -0.0789,  0.3371, -0.2643,  0.485 , -0.384 ],
       [-0.593 ,  0.3678,  1.    , -0.3317,  0.102 , -0.3785,  0.4811,  0.2561,  0.3855, -0.1261],
       [ 0.2698,  0.1036, -0.3317,  1.    ,  0.248 ,  0.2302,  0.0207, -0.6127,  0.2957, -0.1031],
       [ 0.0679,  0.0957,  0.102 ,  0.248 ,  1.    , -0.0462,  0.0432, -0.215 ,  0.495 , -0.1981],
       [ 0.349 , -0.0789, -0.3785,  0.2302, -0.0462,  1.    , -0.2407, -0.278 ,  0.0893,  0.156 ],
       [-0.7985,  0.3371,  0.4811,  0.0207,  0.0432, -0.2407,  1.    ,  0.0096,  0.2185,  0.077 ],
       [-0.1988, -0.2643,  0.2561, -0.6127, -0.215 , -0.278 ,  0.0096,  1.    , -0.2101,  0.373 ],
       [-0.0798,  0.485 ,  0.3855,  0.2957,  0.495 ,  0.0893,  0.2185, -0.2101,  1.    , -0.1276],
       [-0.0217, -0.384 , -0.1261, -0.1031, -0.1981,  0.156 ,  0.077 ,  0.373 , -0.1276,  1.    ]])

In [14]:
selected = np.asarray([0, 1])
U = np.linalg.svd(raw_data[:, selected], full_matrices=0)[0][:, 0]
XU = raw_data.T @ U
raw_data_residual = raw_data - U[:, None] * XU.T
sigma_residual = raw_data_residual.T @ raw_data_residual

In [15]:
import tensorflow as tf

In [121]:
@tf.function
def ipsen_nonlinear(sigma_residual, v, S):
    # Numerator: Use S (ideally symmetric, rank-one, {0,1}^{n,n}) to shield
    # the entries in S that ought to be removed rows/columns (Hadamard product).
    # S with a support of the same k rows/columns can be multiplied directly
    # against v.
    # Denominator: Validate this further. Maybe there is something fishy going
    # on so that the Hessian is almost PSD, but not quite.
    return tf.linalg.norm(
        tf.linalg.matmul(sigma_residual * S, v[:, None])
    ) / tf.math.sqrt(
        tf.math.reduce_sum(S * v * tf.transpose(v))
    )

In [115]:
ipsen_nonlinear(tf.constant(sigma_residual, tf.float64), tf.constant(XU, tf.float64), tf.cast(tf.eye(10), tf.float64))

<tf.Tensor: shape=(), dtype=float64, numpy=0.516555506269217>

In [116]:
@tf.function
def ipsen_nonlinear_hessian(sigma_residual, v, S):
    return tf.hessians(ipsen_nonlinear(sigma_residual, v, S), S)

In [100]:
select_v = np.asarray([1., 1., 0., 0., 0., 1., 0., 1., 0., 0.,])[:, None]
S = select_v @ select_v.T

In [117]:
select_v = np.random.uniform(size=[10, 1])
S = select_v @ select_v.T

In [118]:
h = ipsen_nonlinear_hessian(tf.constant(sigma_residual, tf.float64), tf.constant(XU, tf.float64), tf.constant(S, tf.float64))[0].numpy()
h

array([[[[ 0.0966, -0.0936,  0.0236, ..., -0.0032, -0.0238, -0.0153],
         [ 0.0034, -0.0005,  0.0013, ..., -0.0001, -0.0002, -0.0002],
         [ 0.0014,  0.0015,  0.0005, ...,  0.    ,  0.0002,  0.0001],
         ...,
         [ 0.0029,  0.0001,  0.0021, ..., -0.0003, -0.0002, -0.0005],
         [ 0.002 ,  0.0009,  0.0004, ..., -0.    , -0.0007,  0.0001],
         [ 0.0029,  0.    ,  0.0008, ..., -0.0001,  0.0001, -0.0018]],

        [[-0.0936,  0.096 , -0.0221, ...,  0.0032,  0.0242,  0.0155],
         [ 0.0053, -0.003 ,  0.0016, ..., -0.0001, -0.0008, -0.0006],
         [ 0.001 ,  0.0013, -0.    , ...,  0.    ,  0.0001,  0.0001],
         ...,
         [ 0.0041, -0.0018,  0.0033, ..., -0.0006, -0.0009, -0.0012],
         [ 0.0023,  0.    , -0.0001, ..., -0.0001, -0.0019,  0.    ],
         [ 0.0042, -0.0019,  0.0007, ..., -0.0003, -0.0001, -0.0038]],

        [[ 0.0236, -0.0221,  0.0059, ..., -0.0008, -0.0056, -0.0036],
         [ 0.0021, -0.0006,  0.0007, ..., -0.    , -0.0002

In [120]:
# Hessian is not positive semidefinite nor negative semidefinite.
np.linalg.eigvalsh(h.reshape([100, 100]))

array([-0.0282, -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    ,
       -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    ,
       -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    , -0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.0521,  0.0718,  0.13  ,  0.1321,  0.1508,  0.1933,  0.2121,  0.222 ,  0.2382,  0.2456])